In [1]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import itertools
import copy

In [2]:
from math import radians, cos, sin, asin, sqrt
AVG_EARTH_RADIUS = 6373

def haversine(point1, point2):

    # извлекаем долготу и широту
    lat1, lng1 = point1
    lat2, lng2 = point2

    # переводим все эти значения в радианы
    lat1, lng1, lat2, lng2 = map(radians, (lat1, lng1, lat2, lng2))

    # вычисляем расстояние по формуле
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = sin(lat * 0.5) ** 2 + cos(lat1) * cos(lat2) * sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * asin(sqrt(d))
    return h 

import numpy as np
def haversine_np(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6373 * c
    return km


In [5]:
points = pd.read_csv('points.txt', header=None, names=["raw_data"])

points[['Latitude', 'raw_data']]= points["raw_data"].str.split(n = 1, expand = True)
points[['Longitude', 'raw_data']]= points["raw_data"].str.split(n = 1, expand = True)
points.rename(columns={'raw_data': 'city'}, inplace=True)
points = points.astype({'Latitude': 'float', 
                        'Longitude': 'float'})

# # Делает город индексом #
# points.index=points.city
# del points["city"]|
# points.index.name = None
graph

,node1,node2,color,distance
0,Турнюева,Катангар Лесоучастоя,red,12288.863923
1,Турнюева,Ялунмнское,red,1694.834365
2,Большой Црала,Ялунмнское,red,8204.927271
3,Встеря-3,Новый Цультем,red,13135.873075
4,Встеря-3,Афанашьевский,red,12318.166141
...,...,...,...,...
9994,Туркверов,Балчиклж,red,2799.059563
9995,Киндсево,Хцдатово,red,8939.269707
9996,Сутдево,Мухон-Булык,red,3461.928912
9997,Хлександрова Коса,Спколка,red,12061.447078


In [6]:
graph = pd.read_csv('graph.txt', header=None, index_col=None, names=["raw_data"])

graph[['node1', 'node2']] = graph["raw_data"].str.split(" --- ", n = 1, expand = True)
del graph['raw_data']
graph["color"] = "red"
# graph["estimate"] = 1
# graph["trail"] = 0.18
# graph["distance"] = 10

# добавляю дистанцию
graph = pd.merge(graph, points, right_on="city", left_on="node1")
del graph['city']
graph = pd.merge(graph, points, right_on="city", left_on="node2", suffixes=('_node1', "_node2"))
del graph['city']
graph['distance'] = haversine_np(
    graph['Latitude_node1'],  graph['Longitude_node1'], 
    graph['Latitude_node2'],  graph['Longitude_node2']
)
del graph['Latitude_node1'],  graph['Longitude_node1'], 
del graph['Latitude_node2'],  graph['Longitude_node2']
points

,city,Latitude,Longitude
0,Зарфинские Выселки,70.302069,-93.181730
1,Елеовый Куст,-48.332917,67.878950
2,Буггаковский,4.709965,142.617975
3,Большой Макаеелем,-86.139242,-66.493468
4,Сафрохов,-28.994900,-138.715208
...,...,...,...
9995,Числавль,-80.890212,-130.446348
9996,Горькодская,16.546005,15.222159
9997,Заеаменье,-38.374786,-148.264253
9998,Кэпытова,89.818650,39.711307


In [7]:
G = nx.Graph()
for _, row in graph.iterrows():
    G.add_edge(
        row[0],  # from point
        row[1],  # to point
        attr_dict=row[2:].to_dict()
    )

In [8]:
attrs = {
    row['city']: row[1:].to_dict()
    for _, row in points.iterrows()
}
nx.set_node_attributes(G, attrs)
f"Всего узлов {G.number_of_nodes()}, рёбер {G.number_of_edges()}"

'Всего узлов 8675, рёбер 9999'